In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

In [3]:
SIMULATION_OUTPUT="/mnt/Archivos/Nextcloud/MaestriaPUCP/Thesis/development/project"

In [4]:
RELATIVE_ADEXL_FOLDER="tesis_2020/sar_architecture_cm_differential_v2_sim_neural/adexl/documents"
files_summary = ["Interactive.1.csv", "Interactive.0.csv"]

In [5]:
files_summary_paths = [ Path(SIMULATION_OUTPUT)/RELATIVE_ADEXL_FOLDER/ file_name for file_name in files_summary]

In [6]:
import pandas as pd

In [7]:
def get_df(filepath):
    df = pd.read_csv(filepath, comment='"', header=0)
    return df

In [8]:
df = get_df(files_summary_paths[0])
df.head()

,Point,Test,Output,Nominal,Spec,Weight,Pass/Fail
0,1,tesis_2020:sar_architecture_cm_differential_v2...,/Vin,NaN,NaN,NaN,NaN
1,1,tesis_2020:sar_architecture_cm_differential_v2...,/START_n,NaN,NaN,NaN,NaN
2,1,tesis_2020:sar_architecture_cm_differential_v2...,/RESET_n,NaN,NaN,NaN,NaN
3,1,tesis_2020:sar_architecture_cm_differential_v2...,/ZOH_OUT,NaN,NaN,NaN,NaN
4,1,tesis_2020:sar_architecture_cm_differential_v2...,/I61/VDD,NaN,NaN,NaN,NaN


In [9]:
def extract_parameter(line):
    return line.split(",")[0].strip("\"").split("=")[1].strip("\"")

def extract_index_number(line):
    return int(line.split(",")[0])

In [10]:
def get_metadata_dict(filepath):    
    meta_list=[]
    with open(filepath,'r') as f:
        savenext=False
        meta_pair=[]
        for line in f:
            if(savenext):
                meta_pair.append(extract_index_number(line))
                meta_list.append(meta_pair)
                savenext=False
                continue
            if(line.startswith("\"")):
                meta_pair=[extract_parameter(line)]
                savenext=True

    l=list(zip(*meta_list))
    l.reverse()
    return dict(zip(*l))

In [11]:
def process_file(filepath):
    df = get_df(filepath)
    map_point_to_filename = get_metadata_dict(filepath)
    df["filename"]=df["Point"].map(map_point_to_filename)
    return df

In [12]:
df_list =[ process_file(filepath) for filepath in files_summary_paths]

In [13]:
ndf = pd.concat(df_list)

In [14]:
ndf.dropna(subset=["Nominal"],inplace=True)

In [15]:
from quantiphy import Quantity 

In [16]:
sdf = ndf.groupby("Output").agg({"Nominal":"mean"}).reset_index()
sdf.drop(sdf.loc[sdf["Output"].isin(["t_start","t_interval","t_stop"])].index,inplace=True)
sdf["Nominal"]=sdf["Nominal"].apply(lambda x: str(Quantity(x)))
sdf

,Output,Nominal
0,Average_current_comparator,61.746n
1,Average_current_dac,10.021n
2,Average_current_logic,535.56n
3,Average_current_register,4.3016n
4,Average_current_total,875.48n
5,Average_power_total,1.0468u


In [17]:
RELATIVE_ADEXL_FOLDER="tesis_2020/lcadc_architecture_comp_hysteresis_mixed_reset_sim_neural/adexl/documents/"
files_summary = ["Interactive.0.csv"]

In [18]:
files_summary_paths = [ Path(SIMULATION_OUTPUT)/RELATIVE_ADEXL_FOLDER/ file_name for file_name in files_summary]

In [19]:
df_list =[ process_file(filepath) for filepath in files_summary_paths]
ndf = pd.concat(df_list)
ndf.dropna(subset=["Nominal"],inplace=True)
sdf=ndf.groupby("Output").agg({"Nominal":"mean"}).reset_index()
sdf.drop(sdf.loc[sdf["Output"].isin(["t_start","t_interval","t_stop"])].index,inplace=True)
sdf["Nominal"]=sdf["Nominal"].apply(lambda x: str(Quantity(x)))
sdf

,Output,Nominal
0,average_current_comparator_cc,533.96n
1,average_current_comparator_cr,1.2558u
2,average_current_counter,6.9221n
3,average_current_dac,306.11p
4,average_current_logic_dac,8.6912n
5,average_current_logic_rlc,53.429n
6,average_current_multiplexer,34.586p
7,average_total_current,3.205u
8,average_total_power,3.8459u
